In [37]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

In [38]:
data_dir = 'C:\\Users\\Selva\\Desktop\\skillcraft\\test' 
categories = os.listdir(data_dir)
num_classes = len(categories) 


In [39]:
img_size = 64

In [40]:
def load_data(data_dir, img_size):
    data = []
    labels = []

In [41]:
    for category in categories:
        path = os.path.join(data_dir, category)
        class_num = categories.index(category)
       

In [45]:
def load_data(data_dir, img_size):
    data = []
    labels = []
    
    for category in categories:
        path = os.path.join(data_dir, category)
        class_num = categories.index(category)
        
        for img in os.listdir(path):
            try:
                img_array = tf.keras.preprocessing.image.load_img(os.path.join(path, img), target_size=(img_size, img_size))
                img_array = tf.keras.preprocessing.image.img_to_array(img_array)
                data.append(img_array)
                labels.append(class_num)
            except Exception as e:
                print(e)
                
    data = np.array(data) / 255.0
    labels = np.array(labels)
    return data, labels


In [46]:
data, labels = load_data(data_dir, img_size)

In [47]:
train_data, val_data, train_labels, val_labels = train_test_split(data, labels, test_size=0.2, random_state=42)

In [48]:
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model

In [49]:
base_model = MobileNetV2(input_shape=(img_size, img_size, 3), include_top=False, weights='imagenet')

C:\Users\Selva\AppData\Local\Temp\ipykernel_19184\4125065372.py:1: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  base_model = MobileNetV2(input_shape=(img_size, img_size, 3), include_top=False, weights='imagenet')


9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 39s 4us/step


In [50]:
base_model.trainable = False

In [51]:
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
predictions = Dense(num_classes, activation='softmax')(x)

In [52]:
model = Model(inputs=base_model.input, outputs=predictions)

In [53]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [54]:
datagen = ImageDataGenerator(
    rotation_range=10,
    zoom_range=0.1,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True
)

In [55]:
history = model.fit(datagen.flow(train_data, train_labels, batch_size=32),
                    epochs=10,
                    validation_data=(val_data, val_labels))

Epoch 1/10


c:\Users\Selva\anaconda3\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:120: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


150/150 ━━━━━━━━━━━━━━━━━━━━ 18s 73ms/step - accuracy: 0.6982 - loss: 1.0917 - val_accuracy: 0.9758 - val_loss: 0.1283
Epoch 2/10
150/150 ━━━━━━━━━━━━━━━━━━━━ 11s 74ms/step - accuracy: 0.9624 - loss: 0.1307 - val_accuracy: 0.9525 - val_loss: 0.1511
Epoch 3/10
150/150 ━━━━━━━━━━━━━━━━━━━━ 10s 68ms/step - accuracy: 0.9694 - loss: 0.1007 - val_accuracy: 0.9675 - val_loss: 0.0887
Epoch 4/10
150/150 ━━━━━━━━━━━━━━━━━━━━ 9s 56ms/step - accuracy: 0.9720 - loss: 0.0810 - val_accuracy: 0.9533 - val_loss: 0.1302
Epoch 5/10
150/150 ━━━━━━━━━━━━━━━━━━━━ 8s 53ms/step - accuracy: 0.9710 - loss: 0.0817 - val_accuracy: 0.9842 - val_loss: 0.0528
Epoch 6/10
150/150 ━━━━━━━━━━━━━━━━━━━━ 8s 52ms/step - accuracy: 0.9781 - loss: 0.0592 - val_accuracy: 0.9692 - val_loss: 0.0894
Epoch 7/10
150/150 ━━━━━━━━━━━━━━━━━━━━ 8s 52ms/step - accuracy: 0.9817 - loss: 0.0522 - val_accuracy: 0.9758 - val_loss: 0.0562
Epoch 8/10
150/150 ━━━━━━━━━━━━━━━━━━━━ 8s 53ms/step - accuracy: 0.9791 - loss: 0.0606 - val_accuracy: 0.

In [57]:
import cv2

In [59]:
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break
    frame_resized = cv2.resize(frame, (img_size, img_size))
    frame_normalized = frame_resized / 255.0
    frame_expanded = np.expand_dims(frame_normalized, axis=0)
    
    predictions = model.predict(frame_expanded)
    predicted_class = categories[np.argmax(predictions)]
    
    cv2.putText(frame, f'Gesture: {predicted_class}', (10, 30), 
                cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)
    
    cv2.imshow('Hand Gesture Recognition', frame)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 77ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step
1/1 ━━━━━━━━

In [60]:
val_loss, val_acc = model.evaluate(val_data, val_labels)
print(f'Validation accuracy: {val_acc * 100:.2f}%')

38/38 ━━━━━━━━━━━━━━━━━━━━ 2s 48ms/step - accuracy: 0.9775 - loss: 0.0679
Validation accuracy: 97.75%
